In [ ]:
%%bash
conda install pytorch=1.10.0 torchvision torchaudio=0.10.0 cudatoolkit=11.1 -c pytorch -c conda-forge
cd /root/wenet
git clone https://github.com/wenet-e2e/wenet.git
cd wenet
pip install -r requirements.txt

In [3]:
%cd "/root/wenet/wenet/examples/librispeech/s0/"
%env DATA_DIR=/root/wenet/data/training


/root/wenet/wenet/examples/librispeech/s0
env: DATA_DIR=/root/wenet/data/training


In [ ]:
# prepare dataset
!chmod 777 *.sh
!cp run.sh run-bak.sh
#mkdir export
!mkdir -p $DATA_DIR

In [ ]:
%%bash
# install flac
wget https://downloads.xiph.org/releases/flac/flac-1.3.2.tar.xz
xz -d flac-1.3.2.tar.xz && tar -xvf flac-1.3.2.tar --no-same-owner && cd flac-1.3.2 && ./configure --prefix=/usr --disable-thorough-tests && make && make install

In [ ]:
%%bash
#Data download
./run.sh --stage -1 --stop_stage -1 --datadir $DATA_DIR

In [ ]:
%%bash
# Data preparep
./run.sh --stage 0 --stop_stage 0 --datadir $DATA_DIR

In [4]:
%%bash
# Feature Generation: 
./run.sh --stage 1 --stop_stage 1 --datadir $DATA_DIR

#chmod -R 777 data

stage 1: Feature Generation
using resample and new sample rate is 16000
[2022-12-01 04:13:06.224 pytorch-1-12-cpu-py3-ml-c5-4xlarge-48f6c7eb29a07ab0f18a4946953d:4126 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-12-01 04:13:06.446 pytorch-1-12-cpu-py3-ml-c5-4xlarge-48f6c7eb29a07ab0f18a4946953d:4126 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
dictionary: data/lang_char/train_clean_100_unigram5000_units.txt


processed 1000 wavs, 1266562 frames
processed 2000 wavs, 2521275 frames
processed 3000 wavs, 3774702 frames
processed 4000 wavs, 5044147 frames
processed 5000 wavs, 6313428 frames
processed 6000 wavs, 7586115 frames
processed 7000 wavs, 8860718 frames
processed 8000 wavs, 10142828 frames
processed 9000 wavs, 11413681 frames
processed 10000 wavs, 12684987 frames
processed 11000 wavs, 13930836 frames
processed 12000 wavs, 15197649 frames
processed 13000 wavs, 16483920 frames
processed 14000 wavs, 17756963 frames
processed 15000 wavs, 19013510 frames
processed 16000 wavs, 20269998 frames
processed 17000 wavs, 21535826 frames
processed 18000 wavs, 22812304 frames
processed 19000 wavs, 24074440 frames
processed 20000 wavs, 25333124 frames
processed 21000 wavs, 26613578 frames
processed 22000 wavs, 27891132 frames
processed 23000 wavs, 29177650 frames
processed 24000 wavs, 30439073 frames
processed 25000 wavs, 31706880 frames
processed 26000 wavs, 32996103 frames
processed 27000 wavs, 342480

In [8]:
%%bash
./run.sh --stage 2 --stop_stage 2 --datadir $DATA_DIR
    
#chmod -R 777 data

dictionary: data/lang_char/train_clean_100_unigram5000_units.txt
stage 2: Dictionary and Json Data Preparation
5002 data/lang_char/train_clean_100_unigram5000_units.txt


sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=data/lang_char/input.txt --vocab_size=5000 --model_type=unigram --model_prefix=data/lang_char/train_clean_100_unigram5000 --input_sentence_size=100000000
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/lang_char/input.txt
  input_format: 
  model_prefix: data/lang_char/train_clean_100_unigram5000
  model_type: UNIGRAM
  vocab_size: 5000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 100000000
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_v

In [6]:
%%bash
./run.sh --stage 3 --stop_stage 3 --datadir $DATA_DIR
    
#chmod -R 777 data

dictionary: data/lang_char/train_clean_100_unigram5000_units.txt
Prepare data, prepare required format


In [ ]:
import boto3
import sagemaker
import os
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()

bucket = sess.default_bucket() 
bucket

In [ ]:
%env BUCKET=sagemaker-us-east-1-348052051973
%env PREFIX=wenet

In [ ]:
%%bash
#upload dataset+etc to s3 bucket
aws s3 cp --recursive  exp s3://$BUCKET/$PREFIX/exp
aws s3 cp --recursive  data s3://$BUCKET/$PREFIX/data
#make sure directory data/train_960 exist


In [3]:
%%markdown
We could use terminal with following commands to speed up copying to s3. Replace sagemaker-us-east-1-348052051973 with your owne s3 bucket
nohup aws s3 sync ~/wenet/data/training/LibriSpeech/dev-clean s3://sagemaker-us-east-1-348052051973/wenet/export/LibriSpeech/dev-clean --only-show-errors &

nohup aws s3 sync ~/wenet/data/training/LibriSpeech/dev-other s3://sagemaker-us-east-1-348052051973/wenet/export/LibriSpeech/dev-other --only-show-errors &

nohup aws s3 sync ~/wenet/data/training/LibriSpeech/test-clean s3://sagemaker-us-east-1-348052051973/wenet/export/LibriSpeech/test-clean --only-show-errors &

nohup aws s3 sync ~/wenet/data/training/LibriSpeech/test-other s3://sagemaker-us-east-1-348052051973/wenet/export/LibriSpeech/test-other --only-show-errors &

nohup aws s3 sync ~/wenet/data/training/LibriSpeech/train-clean-100 s3://sagemaker-us-east-1-348052051973/wenet/export/LibriSpeech/train-clean-100 --only-show-errors &

nohup aws s3 sync ~/wenet/data/training/LibriSpeech/train-clean-360 s3://sagemaker-us-east-1-348052051973/wenet/export/LibriSpeech/train-clean-360 --only-show-errors &

nohup aws s3 sync ~/wenet/data/training/LibriSpeech/train-other-500 s3://sagemaker-us-east-1-348052051973/wenet/export/LibriSpeech/train-other-500 --only-show-errors &

nohup aws s3 sync ~/wenet/data/training/LibriSpeech/dev-clean s3://sagemaker-us-east-1-348052051973/wenet/export/Librispeech/dev-clean --only-show-errors &

nohup aws s3 sync ~/wenet/data/training/LibriSpeech/dev-other s3://sagemaker-us-east-1-348052051973/wenet/export/Librispeech/dev-other --only-show-errors &

nohup aws s3 sync ~/wenet/data/training/LibriSpeech/test-clean s3://sagemaker-us-east-1-348052051973/wenet/export/Librispeech/test-clean --only-show-errors &

nohup aws s3 sync ~/wenet/data/training/LibriSpeech/test-other s3://sagemaker-us-east-1-348052051973/wenet/export/Librispeech/test-other --only-show-errors &

nohup aws s3 sync ~/wenet/data/training/LibriSpeech/train-clean-100 s3://sagemaker-us-east-1-348052051973/wenet/export/Librispeech/train-clean-100 --only-show-errors &

nohup aws s3 sync ~/wenet/data/training/LibriSpeech/train-clean-360 s3://sagemaker-us-east-1-348052051973/wenet/export/Librispeech/train-clean-360 --only-show-errors &

nohup aws s3 sync ~/wenet/data/training/LibriSpeech/train-other-500 s3://sagemaker-us-east-1-348052051973/wenet/export/Librispeech/train-other-500 --only-show-errors &
